# Filtrado colaborativo

El filtrado colaborativo es una de las técnicas más usadas para construir sistemas de recomendación inteligentes, que mejoran sus recomendaciones a medida que los usuarios interaccionan con el sistema.

La mayoría de sitios web como Amazon, Netflix, Youtube y Spotify usan filtrado colaborativo como parte de sus sistemas de recomendación.

## Definición

Técnica que permite identificar items que potencialmente pueden interesar a un determinado usuarios, basándonos en las reacciones de usuarios similares.

Los algoritmos de filtrado colaborativo tienen 2 fases:
 - Encontrar un subconjunto de usuarios con gustos similares al usuario para el cual se quiere realizar la recomendación
 - Combinar los valores de los usuarios similares para generar una lista de recomendaciones
 
Normalmente los datos para construir un sistema de recomendación se almacenan en una matriz donde las columnas representan un conjunto de items y las filas los distintos usuarios, que reaccionan ante algunos de los items:

![](../img/rating-matrix.webp)

Las reacciones de los usuarios pueden ser:

- **explícitas**: valorar una película, darle a like a una canción. En este caso el usuario indica directamente su preferencia por el item, en ocasiones en una escala numérica (por ej. 1-5)
- **implícitas**: ver una película, escuchar una canción. En este caso inferimos que el usuario está interesado en el item por la propia interacción, aunque puede no ser el caso

El objetivo del filtrado colaborativo es completar la matriz de usuarios-items, infiriendo las preferencias de ciertos items a los cuales el usuario no ha reaccionado todavía. Este problema se conoce con el nombre de [*matrix completion*](https://en.wikipedia.org/wiki/Matrix_completion) en la literatura y tiene diversas aplicaciones.

## Tipos de filtrado colaborativo

De forma similar al clustering, no existe un único algoritmo del filtrado colaborativo, sino que hay toda una familia de algoritmos dependiendo de las respuestas que demos a las siguientes preguntas:

- ¿Cómo se identifican que usuarios o items son similares entre si?
- ¿Cómo se combinan las reacciones de usuarios similares para estimar la reacción de un usuario con un item determinado?

Es importante destacar que, en una aproximación basada únicamente en filtrado colaborativo, la similaridad entre usuarios únicamente se calcula basándonos en sus interacciones, y no en otro tipo de variables externas como edad, sexo, etc.

Según la implementación, también se pueden distinguir dos tipos de filtrado colaborativo:

- Basado en **memoria**: las recomendaciones se realizan accediendo a la base de datos de usuarios-items por completo y de forma directa. La ventaja principal es que se pueden adaptar a cambios en los datos.

- Basado en **modelos**: en este caso se entrena un modelo a partir de la matriz anterior, que luego se usa para realizar las recomendaciones. La principal ventaja es que suelen ser bastante más eficientes computacionalmente hablando.

## FC basado en memoria

### Medidas de distancia

El primer paso es definir una distancia para encontrar usuarios similares. Vamos a trabajar con los siguientes datos de prueba:

In [1]:
import pandas as pd
ratings = pd.DataFrame(
    [[1, 2], [2, 4], [2.5, 4], [4.5, 5]], 
    columns=["item1", "item2"], 
    index=["A", "B", "C", "D"]
)
ratings

,item1,item2
A,1.0,2
B,2.0,4
C,2.5,4
D,4.5,5


In [2]:
from sklearn.metrics import pairwise_distances

pairwise_distances(ratings, metric="euclidean")

array([[0.        , 2.23606798, 2.5       , 4.60977223],
       [2.23606798, 0.        , 0.5       , 2.6925824 ],
       [2.5       , 0.5       , 0.        , 2.23606798],
       [4.60977223, 2.6925824 , 2.23606798, 0.        ]])

En este ejemplo, usando la distancia Euclidea, vemos que el usuario C está más próximo a D (distancia 2.23) que a A (distancia 2.5). Sin embargo, para este tipo de aplicaciones, muchas veces nos interesa más los patrones de los ratings para todos los items que los valores absolutos en si.

En ese caso, podemos usar otra medida de distancia como la correlación o la [distancia coseno](https://en.wikipedia.org/wiki/Cosine_similarity):

$$1 - \frac{u \cdot v}{||u||_2 ||v||_2}$$

In [3]:
from scipy.spatial.distance import cosine

pairwise_distances(ratings, metric=cosine)

array([[0.        , 0.        , 0.00450453, 0.03600738],
       [0.        , 0.        , 0.00450453, 0.03600738],
       [0.00450453, 0.00450453, 0.        , 0.01513723],
       [0.03600738, 0.03600738, 0.01513723, 0.        ]])

Usando esta distancia el usuario C está más próximo a A (distancia 0.004) que a D (0.015). Además, los usuarios A y B son idénticos, ya que aunque no tienen las mismas valoraciones siguen la misma tendencia a la hora de valorar los dos items.

Puesto que distintas personas son más estrictas que otras a la hora de valorar, es habitual restar la media de todas las interacciones para un usuario determinado para eliminar dichos sesgos:

In [4]:
ratings.sub(ratings.mean(axis=1), axis=0)

,item1,item2
A,-0.50,0.50
B,-1.00,1.00
C,-0.75,0.75
D,-0.25,0.25


### Combinar interacciones

Una vez ordenados los usuarios de menor a mayor respecto a la distancia con el usuario target, hay que decidir cuantos de ellos se van a usar para predecir la interacción. A continuación, tenemos que combinar sus interacciones. La forma más sencilla es usando la media.

Otra opción es usar una media ponderada por el inverso de la distancia (similaridad). De esta forma las interacciones de usuarios similares tienen más peso en la estimación.

### Tipos

El ejemplo anterior se trata de filtrado colaborativo basado en usuario. Sin embargo, dependiendo de la aplicación y del número distinto de usuarios/items, puede ser mejor realizar una aproximación basada en items:

- Basado en **usuario**: para un usuario U se buscan usuarios similares y se determina el rating de un item I como la combinación de los ratings de los usuarios similares

- Basado en **item**: si el usuario U no ha evaluado el item I, estimamos esta interacción combinando las evaluaciones de ese mismo usuario para los N items más similares

Ambas aproximaciones son muy similares y únicamente cambia el punto de vista. Los sistema de recomendación basados en items suelen funcionar mejor en entornos donde hay muchos más items que usuarios, por ejemplo Amazon. Sin embargo no suelen ser la mejor opción para conjuntos de datos de ratings, ya que las recomendaciones únicamente están basadas en el propio usuario y suelen ser muy obvias.


#### Ejemplo con datos artificiales

Vamos a usar la libreria [Surprise](http://surpriselib.com/)

Otra librería que implementa este tipo de técnicas es [`lighfm`](https://github.com/lyst/lightfm)

In [5]:
# Para instalar la libreria hay que descomentar esta linea
#%conda install -c conda-forge scikit-surprise

In [6]:
from surprise import Dataset
from surprise import Reader

ratings_dict = {
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))

# Loads Pandas dataframe
data = Dataset.load_from_df(df, reader)
df

,item,user,rating
0,1,A,1.0
1,2,A,2.0
2,1,B,2.0
3,2,B,4.0
4,1,C,2.5
5,2,C,4.0
6,1,D,4.5
7,2,D,5.0
8,1,E,3.0


In [7]:
from surprise import KNNWithMeans

sim_options = {
    "name": "cosine",
    "user_based": True,
}

knn = KNNWithMeans(sim_options=sim_options)

trainingSet = data.build_full_trainset()
knn.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [8]:
pred = knn.predict('E', 2)
pred.est

3.111111111111111

#### Ejercicio

Vamos a usar los datos de ratings de canciones de [Kaggle](https://www.kaggle.com/rymnikski/dataset-for-collaborative-filters):

1. Cargar los datos como un DataFrame de pandas

2. ¿Cuantos usuarios e items distintos hay?

3. Calcula la media, máximo y mínimo número de items que han evaluado los usuarios

4. Entrena un algoritmo de filtrado colaborativo `KNNWithMeans`

## FC basado en modelos

Estos algoritmos se basan en reducir o comprimir la matriz de usuarios-items. Esto es útil ya que en la práctica es muy habitual que la matriz sea muy dispersa, es decir, que muchos usuarios no hayan interaccionado con muchos de los items.

La aproximación más popular consiste en reducir la dimensionalidad de la matriz usando técnicas de factorización de matrices. Dada la matriz original con $m$ usuarios y $n$ items $A \in R^{m \times n}$, el modelo aprende:

- Una matriz de embedding de los usuarios, $U \in R^{m \times d}$, donde la fila $i$ es el embedding del usuario $i$

- Una matriz de embedding de los items, $V \in R^{n \times d}$, donde la fila $j$ es el embedding del item $j$ 

![](../img/model_based_cf.png)

Fuente: [Matrix Factorization](https://developers.google.com/machine-learning/recommendation/collaborative/matrix)

Los embeddings se aprenden de forma que el producto $UV^T$ es una buena aproximación de la matriz original $A$. Es importante destacar que la posición $(i, j)$ de la matriz $UV^T$ es simplemente el producto de los embeddings del usuario $i$ y del item $j$, que debería de ser aproximadamente $A_{i,j}$

Esta nueva representación es mucho más compacta que la matriz original:

 - $A$ tiene $n \times m$ elementos (ignorando el hecho que los elementos que faltan no es necesario almacenarlos)
 
 - $U$ y $V$ tienen $nd + md$ elementos y tipicamente $d$ es mucho menor que $n$ y $m$


Para aprender los embeddings se resuelve el siguiente problema de optimización:

$$\min_{U, V} \sum_{i, j}(A_{i,j} - U_i^T V_j)^2$$

Aunque existen varios algoritmos, uno de los más populares es SVD (*Singular Value Decomposition*)

In [9]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")

param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/alberto/.surprise_data/ml-100k
0.9641043327174748
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


## Ventajas y desventajas del filtrado colaborativo

Ventajas:

- No necesita información sobre los items o los usuarios, únicamente sus interacciones

- No es necesario conocimiento del dominio específico

- El modelo puede ayudar a los usuarios a encontrar nuevos items de interés, evitando especializarse en exceso en el perfil concreto del usuario.

Desventajas:

- Cuando un nuevo item es añadido no se recomiendo hasta que ningún usuario interaccione con el (*cold-start*)

- Si los datos son muy dispersos la calidad de las recomendaciones para determinados usuarios puede ser muy pobre

- No es sencillo incluir otras variables que pueden mejorar las recomendaciones, como por ejemplo el género de las películas

- Escalar estos algoritmos no es sencillo en sistemas con millones de usuarios e items